In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:
torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        #print("data length", len(data))
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #print("output length", len(data))
        #print("target length", len(target))
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=1.6370038986206055 batch_id=1874: 100%|██████████| 1875/1875 [00:16<00:00, 112.22it/s]



Test set: Average loss: 1.6850, Accuracy: 3895/10000 (38.95%)



In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 16, 3) # Output - 26  | RF - 3
        self.norm1 = nn.BatchNorm2d(16)
        self.dropout1 = nn.Dropout(0.1)
        
        self.conv2 = nn.Conv2d(16, 16, 3) # Output - 24 | RF - 5
        self.norm2 = nn.BatchNorm2d(16)
        self.dropout2 = nn.Dropout(0.1)
        
        self.conv3 = nn.Conv2d(16, 16, 3) # Output - 22 | RF - 7
        self.norm3 = nn.BatchNorm2d(16)
        self.dropout3 = nn.Dropout(0.1)
        
        self.pool1 = nn.MaxPool2d(2, 2) # Output - 11  | RF - 14
        
        self.conv4 = nn.Conv2d(16, 16, 3) # Output - 9  | RF - 16
        self.norm4 = nn.BatchNorm2d(16)
        self.dropout4 = nn.Dropout(0.1)
        
        self.conv5 = nn.Conv2d(16, 16, 3)  # Output - 7  | RF - 18
        self.norm5 = nn.BatchNorm2d(16)
        self.dropout5 = nn.Dropout(0.1)
        
        self.conv6 = nn.Conv2d(16, 16, 3)  # Output - 5  | RF - 20
        self.norm6 = nn.BatchNorm2d(16)
        self.dropout6 = nn.Dropout(0.1)
        
        self.conv7 = nn.Conv2d(16, 10, 3) # Output - 3 | RF - 22
        self.gap = nn.AdaptiveAvgPool2d(1)
        
    def forward(self, x):
        x = self.dropout1(self.norm1(F.relu(self.conv1(x))))
        x = self.dropout2(self.norm2(F.relu(self.conv2(x))))
        x = self.dropout3(self.norm3(F.relu(self.conv3(x))))
        x = self.pool1(x)
        x = self.dropout4(self.norm4(F.relu(self.conv4(x))))
        x = self.dropout5(self.norm5(F.relu(self.conv5(x))))
        x = self.dropout6(self.norm6(F.relu(self.conv6(x))))
        x = self.conv7(x)
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        return F.log_softmax(x)
#F.relu(self.conv3(x))

In [8]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,320
       BatchNorm2d-5           [-1, 16, 24, 24]              32
           Dropout-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 16, 22, 22]           2,320
       BatchNorm2d-8           [-1, 16, 22, 22]              32
           Dropout-9           [-1, 16, 22, 22]               0
        MaxPool2d-10           [-1, 16, 11, 11]               0
           Conv2d-11             [-1, 16, 9, 9]           2,320
      BatchNorm2d-12             [-1, 16, 9, 9]              32
          Dropout-13             [-1, 16, 9, 9]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [9]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print("epoch: ", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/1875 [00:00<?, ?it/s]

epoch:  1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0492, Accuracy: 9844/10000 (98.44%)

epoch:  2


  0%|          | 0/1875 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0225, Accuracy: 9937/10000 (99.37%)

epoch:  18


  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0212, Accuracy: 9934/10000 (99.34%)

epoch:  19


loss=0.0003806948661804199 batch_id=1874: 100%|██████████| 1875/1875 [00:10<00:00, 180.91it/s] 



Test set: Average loss: 0.0172, Accuracy: 9947/10000 (99.47%)



In [ ]:
10- 99.26, 99.34, 99.32, 99.37, 99.29, 99.33

In [ ]:
[0.1125, 0.125, 0.5]  - 99.29
[0.13, 0.16]  - 99.30

In [40]:
class Net(nn.Module):
    def __init__(self):
        """ This function instantiates all the model layers """
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 16, 3)  # Input: 28x28x1 | Output: 26x26x16 | RF: 3x3
        self.norm1 = nn.BatchNorm2d(16)
        self.dropout1 = nn.Dropout(0.15)

        self.conv2 = nn.Conv2d(16, 16, 3)  # Input: 26x26x16 | Output: 24x24x16 | RF: 5x5
        self.norm2 = nn.BatchNorm2d(16)
        self.dropout2 = nn.Dropout(0.15)

        self.conv3 = nn.Conv2d(16, 16, 3)  # Input: 24x24x16 | Output: 22x22x32 | RF: 7x7
        self.norm3 = nn.BatchNorm2d(16)

        self.pool = nn.MaxPool2d(2, 2)  # Input: 22x22x32 | Output: 11x11x32 | RF: 14x14

        self.conv4 = nn.Conv2d(16, 16, 3)  # Input: 9x9x16 | Output: 7x7x16 | RF: 18x18
        self.norm4 = nn.BatchNorm2d(16)
        self.dropout4 = nn.Dropout(0.15)

        self.conv5 = nn.Conv2d(16, 16, 3)  # Input: 7x7x16 | Output: 5x5x16 | RF: 20x20
        self.norm5 = nn.BatchNorm2d(16)
        self.dropout5 = nn.Dropout(0.15)

        self.conv6 = nn.Conv2d(16, 10, 3)  # Input: 5x5x16 | Output: 3x3x10 | RF: 22x22

        self.gap = nn.AdaptiveAvgPool2d(1)  # Input: 3x3x10 | Output: 1x1x10
    
    def forward(self, x):
        """ This function defines the network structure """
        net = self.dropout1(self.norm1(F.relu(self.conv1(x))))
        net = self.dropout2(self.norm2(F.relu(self.conv2(net))))
        net = self.pool(self.norm3(F.relu(self.conv3(net))))
        net = self.dropout4(self.norm4(F.relu(self.conv4(net))))
        net = self.dropout5(self.norm5(F.relu(self.conv5(net))))
        net = F.relu(self.conv6(net))
        net = self.gap(net)
        net = net.view(-1, 10)
        return F.log_softmax(net)

In [41]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,320
       BatchNorm2d-5           [-1, 16, 24, 24]              32
           Dropout-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 16, 22, 22]           2,320
       BatchNorm2d-8           [-1, 16, 22, 22]              32
         MaxPool2d-9           [-1, 16, 11, 11]               0
           Conv2d-10             [-1, 16, 9, 9]           2,320
      BatchNorm2d-11             [-1, 16, 9, 9]              32
          Dropout-12             [-1, 16, 9, 9]               0
           Conv2d-13             [-1, 16, 7, 7]           2,320
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [42]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print(f'Epoch {epoch}:')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 1:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0494, Accuracy: 9858/10000 (98.58%)

Epoch 2:


  0%|          | 0/1875 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0203, Accuracy: 9946/10000 (99.46%)

Epoch 6:


  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0219, Accuracy: 9928/10000 (99.28%)

Epoch 7:


loss=0.15513765811920166 batch_id=183:  10%|▉         | 184/1875 [00:00<00:08, 201.20it/s]  


KeyboardInterrupt: 